In [ ]:
# TAKES AUTH_ AS AN INPUT!!!
from htmd.ui import *


In [ ]:
from moleculekit.molecule import Molecule
from moleculekit.tools.voxeldescriptors import getVoxelDescriptors
from moleculekit.tools.atomtyper import prepareProteinForAtomtyping

# First let's read our two favourite protein and ligand structures
prot = Molecule( '3PTB')

prot.filter('protein')

prot = prepareProteinForAtomtyping(prot)
prot_vox, prot_centers, prot_N = getVoxelDescriptors(prot, buffer=8)

In [ ]:
from moleculekit.tools.atomtyper import metal_atypes
from moleculekit.molecule import Molecule
from moleculekit.tools.voxeldescriptors import getVoxelDescriptors
from moleculekit.tools.atomtyper import prepareProteinForAtomtyping
prot = Molecule('5SXR', validateElements=False)

# proof that it takes auth_ version as an input (here auth_seq_id)
print(prot.get('resid', sel='chain A and protein'))

prot = Molecule('1S00', validateElements=False)

# proof that it takes auth_ version as an input (here auth_asym_id)
prot.filter(f'protein')
print(prot.get('chain', sel='protein'))


In [ ]:
import deeplife_utils

prot = Molecule('3wa2', validateElements=False)
chain_id = 'X'
prot.filter(f'chain {chain_id} and protein')
print(len(set(prot.get('resid', sel=f'chain {chain_id} and protein'))))

sequence = ""
previous_id = float('-inf')
for id, residue in zip(prot.get('resid', sel=f'chain {chain_id} and protein'), prot.get('resname', sel=f'chain {chain_id} and protein')):
    if previous_id == id:
        continue
    else:
        sequence += deeplife_utils.three_to_one(residue)
    previous_id = id
print(sequence)
print(len(sequence))

In [2]:
import numpy as np
import os
import json
import deeplife_utils
from moleculekit.tools.atomtyper import metal_atypes
from moleculekit.molecule import Molecule
from moleculekit.tools.voxeldescriptors import getVoxelDescriptors
from moleculekit.tools.atomtyper import prepareProteinForAtomtyping


with open(f'../cryptobench/whole_dataset.json', 'r') as json_file:
    dataset = json.load(json_file)

for apo_structure, holo_structures in dataset.items():
    apo_pockets = set()    
    for holo_structure in holo_structures:
        
        pdb_id = holo_structure['holo_pdb_id']
        chain_id = holo_structure['holo_chain']
        apo_pocket = holo_structure['apo_pocket_selection']
        pocket = holo_structure['holo_pocket_selection']
        id = pdb_id.lower() + chain_id.upper()
        
        # it gets stuck on 5c30. Just skip it, I am too furious to deal with it now
        if apo_structure == '5c30':
            break

        if os.path.isfile(f'../data/holo-voxel-features/{pdb_id.lower()}{chain_id.upper()}.npy') \
            and os.path.isfile(f'../data/holo-voxel-annotations/{pdb_id.lower()}{chain_id.upper()}.npy') \
            and os.path.isfile(f'../data/holo-sequences/{pdb_id.lower()}{chain_id.upper()}.txt'):
            break
        
        new_apo_residues = [residue.split('_')[1] for residue in apo_pocket if residue.split('_')[1] not in apo_pockets]
        # probably a homomer
        if (len(apo_pocket) - len(new_apo_residues)) / len(apo_pocket) > 0.75:
            continue
        apo_pockets.update(new_apo_residues)

        binding_residues = [residue.split('_')[1] for residue in pocket]
        binding_residues_query = f'resid {" or resid ".join(binding_residues)}'
        print(id)
        
        try:
            prot = Molecule(pdb_id, validateElements=False)
            prot.filter(f'chain {chain_id} and protein')
            prot = prepareProteinForAtomtyping(prot)
            prot.filter(f"protein or water or element {' '.join(metal_atypes)}")
    
            prot_voxels, prot_centers, N = getVoxelDescriptors(prot, buffer=8)
            annotation_voxels, _, _ = getVoxelDescriptors(prot, buffer=8, userchannels=prot.atomselect(binding_residues_query).reshape(-1,1))

        except:
            continue
                
        features = prot_voxels.reshape(N[0], N[1], N[2], prot_voxels.shape[1])
        annotations = annotation_voxels.reshape(N[0], N[1], N[2], annotation_voxels.shape[1])
        
        with open(f'../data/holo-voxel-features/{pdb_id.lower()}{chain_id.upper()}.npy', 'wb') as f:
            np.save(f, features)
        with open(f'../data/holo-voxel-annotations/{pdb_id.lower()}{chain_id.upper()}.npy', 'wb') as f:
            np.save(f, annotations)
        with open(f'../data/holo-sequences/{pdb_id.lower()}{chain_id.upper()}.txt', 'w') as f:
            prot = Molecule(pdb_id, validateElements=False)
            sequence = ""
            previous_id = float('-inf')
            for resid, residue in zip(prot.get('resid', sel=f'chain {chain_id} and protein'), prot.get('resname', sel=f'chain {chain_id} and protein')):
                if previous_id == resid:
                    continue
                else:
                    sequence += deeplife_utils.three_to_one(residue)
                previous_id = resid
            f.write(sequence)